## Implementation of Canonical Hopfield Model

### [Hopfield Model Publication](https://www.pnas.org/content/79/8/2554)

#### Based on work from [Chechik et al., 1998.](https://www-mitpressjournals-org.proxy.lib.duke.edu/doi/pdf/10.1162/089976698300017124)

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
M = 100 #number of memories
N = 800 #number of neurons
timesteps = 3 #number of timesteps for the simulation

In [146]:
#creating patterns

patterns = np.empty(shape = (M, N))

for m in range(M):
    patterns[m] = np.random.choice([-1, 1], N)

In [147]:
%%time

#creating the initial connectivity matrix based on a sum of the patterns

connectivity = np.zeros(shape = (N, N))

for m in range(M):
    for i in range(N):
        for j in range(N):
            connectivity[i][j] = connectivity[i][j] + (patterns[m][i] * patterns[m][j])
            connectivity[i][i] = 0
            
Wij = connectivity / np.sqrt(M)

Wall time: 1min 45s


In [148]:
np.unique(Wij)

array([-4.2, -4. , -3.8, -3.6, -3.4, -3.2, -3. , -2.8, -2.6, -2.4, -2.2,
       -2. , -1.8, -1.6, -1.4, -1.2, -1. , -0.8, -0.6, -0.4, -0.2,  0. ,
        0.2,  0.4,  0.6,  0.8,  1. ,  1.2,  1.4,  1.6,  1.8,  2. ,  2.2,
        2.4,  2.6,  2.8,  3. ,  3.2,  3.4,  3.6,  3.8,  4. ,  4.2,  4.4])

In [58]:
#x is our network's activity pattern, this changes over a certain number of timesteps and represents the value of each of the neurons within the second dim

X = np.zeros(shape = (timesteps, N))

In [59]:
#creating a state at X0 that is similar (or in this case the same) to one of the patterns

X[0] = patterns[0]

In [60]:
X[0][5:405] = np.random.choice([-1, 1], 400)

In [61]:
%%time

#ok now lets generalize this function to multiple timesteps, and with every neuron in the network - this is the update rule

for t in range(timesteps - 1):
    X[t+1,:] = np.sign(Wij.dot(X[t,:]))

Wall time: 1 ms


In [62]:
%%time

m_overlap = np.empty(shape = (timesteps, M))
mu = []

for t in range(timesteps):
    for u in range(M):
        mu = []
        for j in range(N):
            mu = np.append(mu, patterns[u][j] * X[t][j])
        m_overlap[t][u] = ((1/N) * mu.sum())
        
        
#create an overlap array that contains each timestep in the first dimension and the overlap value (which I think should be a percentage?) 
#for M number of patterns in the second dimension 

Wall time: 805 ms


In [63]:
for i in range(timesteps):
    print(m_overlap[i][0])

0.47250000000000003
0.9400000000000001
1.0


In [28]:
def hopfield_model_fxn_working(mem, neu, tim = 10):

    M = mem #number of memories
    N = neu #number of neurons
    timesteps = tim #number of timesteps for the simulation


    #creating patterns

    patterns = np.empty(shape = (M, N))
    for m in range(M):
        patterns[m] = np.random.choice([-1, 1], N)


    #creating the initial connectivity matrix based on a sum of the patterns

    connectivity = np.zeros(shape = (N, N))
    for m in range(M):
        for i in range(N):
            for j in range(N):
                connectivity[i][j] = connectivity[i][j] + (patterns[m][i] * patterns[m][j])
                connectivity[i][i] = 0
    Wij = connectivity / np.sqrt(M)

    
    #want an initial overlap of 0.8
    X = np.zeros(shape = (timesteps, N))
    X[0] = patterns[0]
    n_change = int(X[0].shape[0] * 0.2)
#     X[0][1:n_change] = -X[0][1:n_change]

    for t in range(timesteps - 1):
        X[t+1,:] = np.sign(Wij.dot(X[t,:]))

    m_overlap = np.empty(shape = (timesteps, M))
    mu = []

    for t in range(timesteps):
        for u in range(M):
            mu = []
            for j in range(N):
                mu = np.append(mu, patterns[u][j] * X[t][j])
            m_overlap[t][u] = ((1/N) * mu.sum())
    
    #return the value of the corrupted memory 
    return(m_overlap[1][0])

In [26]:
%%time

theo_max = N * .14 #theoretical maximum capacity of the network according to OG publication
mem_tests = 1 #total tests to run
starting_mem_count = int(theo_max - 15) #starting number of memories for testing the model
tim = 2 #number of timesteps, they state in the paper that their overlap was calculated after a single step

output = np.empty(shape = (mem_tests))

for i in range(mem_tests):
    i = i + starting_mem_count
    output[i - starting_mem_count] = hopfield_model_fxn_working(mem = i, neu = 800, tim = 2)

[[ 0.6025  0.015  -0.005  -0.025   0.055  -0.015   0.015   0.0225 -0.0075
   0.01    0.015   0.0075  0.02   -0.0475 -0.05   -0.03    0.0525  0.0475
  -0.0125 -0.035   0.09   -0.0175  0.0225  0.0075  0.      0.0325  0.0525
  -0.0225  0.025  -0.02   -0.0125  0.015  -0.01   -0.04   -0.03    0.005
  -0.025  -0.005   0.03    0.025   0.0025  0.06    0.01    0.0025  0.025
   0.05   -0.055   0.0175 -0.0125 -0.015   0.01    0.035  -0.035   0.0225
  -0.005   0.0475 -0.035   0.0425  0.0075 -0.0325  0.005   0.035   0.05
   0.0175  0.0775 -0.0225 -0.0125  0.0575  0.0025 -0.04    0.02    0.0025
   0.01   -0.025   0.0625  0.0175  0.0325  0.0275  0.03   -0.0575  0.02
   0.02    0.02   -0.1     0.005   0.015  -0.04    0.0325 -0.0675 -0.0075
   0.0075  0.02   -0.0275 -0.02   -0.055  -0.0775 -0.08  ]
 [ 0.9275  0.06   -0.01   -0.045   0.03   -0.065   0.01   -0.0025 -0.0075
   0.02    0.     -0.0175 -0.005  -0.0575 -0.035   0.      0.0725  0.0175
  -0.0225 -0.08    0.105  -0.0025  0.0525 -0.0675  0.02    

In [60]:
def hopfield_model(mem, neu, tim = 2):

    M = mem #number of memories
    N = neu #number of neurons
    timesteps = tim #number of timesteps for the simulation


    #creating patterns

    patterns = np.empty(shape = (M, N))
    for m in range(M):
        patterns[m] = np.random.choice([-1, 1], N)


    #creating the initial connectivity matrix based on a sum of the patterns

    connectivity = np.zeros(shape = (N, N))
    for m in range(M):
        for i in range(N):
            for j in range(N):
                connectivity[i][j] = connectivity[i][j] + (patterns[m][i] * patterns[m][j])
                connectivity[i][i] = 0
    Wij = connectivity / np.sqrt(M)


    #want an initial overlap of 0.8
    X = np.zeros(shape = (timesteps, N))
    X[0] = patterns[0]
    n_change = int(X[0].shape[0] * 0.1)
    X[0][1:n_change] = -X[0][1:n_change]

    for t in range(timesteps - 1):
        X[t+1,:] = np.sign(Wij.dot(X[t,:]))

    m_overlap = np.empty(shape = (timesteps, M))
    mu = []

    for t in range(timesteps):
        for u in range(M):
            mu = []
            for j in range(N):
                mu = np.append(mu, patterns[u][j] * X[t][j])
            m_overlap[t][u] = ((1/N) * mu.sum())
            
    return(m_overlap.take(0, axis = 1))

In [110]:
def hopfield_model_synaptic_deletion(mem, neu, tim = 2, deletion_type = 'min_val', threshold = .5):

    M = mem #number of memories
    N = neu #number of neurons
    timesteps = tim #number of timesteps for the simulation


    #creating patterns
    patterns = np.empty(shape = (M, N))
    for m in range(M):
        patterns[m] = np.random.choice([-1, 1], N)


    #creating the initial connectivity matrix based on a sum of the patterns
    connectivity = np.zeros(shape = (N, N))
    for m in range(M):
        for i in range(N):
            for j in range(N):
                connectivity[i][j] = connectivity[i][j] + (patterns[m][i] * patterns[m][j])
                connectivity[i][i] = 0
    
    Wij = connectivity / np.sqrt(M)
    z = Wij.copy()
    t = threshold
    
    ## min val deletion
    if deletion_type == 'min_val':
        for i in range(len(z[i])):
            for j in range(len(z[j])):
                if np.abs(z[i][j]) > t:
                    z[i][j] = z[i][j]
                elif z[i][j] == t:
                    z[i][j] = 0
                else:
                    z[i][j] = 0
    ##compressed deletion
    if deletion_type == 'compressed':
        for i in range(len(z[i])):
            for j in range(len(z[j])):
                if np.abs(z[i]) <= t:
                    z[i][j] = 0
                elif z[i][j] > t:
                    z[i][j] = z[i][j] - t
                elif z[i][j] < -t:
                    z[i][j] = z[i][j] + t
    ##clipping modification/deletion
    if deletion_type == 'clipping':
        for i in range(len(z[i])):
            for j in range(len(z[j])):
                if z[i][j] > t:
                    z[i][j] = 1
                elif z[i][j] < -t:
                    z[i][j] = -1
                else:
                    z[i][j] = 0
    
    #set the connectivity matrix to the modified/deleted synaptic values
    Wij = z.copy()

    #want an initial overlap of 0.8
    X = np.zeros(shape = (timesteps, N))
    X[0] = patterns[0].copy()
    n_change = int(X[0].shape[0] * 0.1)
    X[0][1:n_change] = -X[0][1:n_change]

    for t in range(timesteps - 1):
        X[t+1,:] = np.sign(Wij.dot(X[t,:]))

    m_overlap = np.empty(shape = (timesteps, M))
    mu = []

    for t in range(timesteps):
        for u in range(M):
            mu = []
            for j in range(N):
                mu = np.append(mu, patterns[u][j] * X[t][j])
            m_overlap[t][u] = ((1/N) * mu.sum())
            
    return(m_overlap.take(0, axis = 1)[1])

In [114]:
np.arange(0, 102, 2)

array([  0,   2,   4,   6,   8,  10,  12,  14,  16,  18,  20,  22,  24,
        26,  28,  30,  32,  34,  36,  38,  40,  42,  44,  46,  48,  50,
        52,  54,  56,  58,  60,  62,  64,  66,  68,  70,  72,  74,  76,
        78,  80,  82,  84,  86,  88,  90,  92,  94,  96,  98, 100])